# Basic example
This is a basic notebook, used only to detect and track objects

In [1]:
import sys
sys.path.insert(0,'../..')
sys.path.insert(0,'..')

import cv2
import glob
import os
import numpy as np
import pickle 

os.environ["CUDA_VISIBLE_DEVICES"]="0"

from random import randint
import matplotlib.pyplot as plt

from pathlib import Path

from time import time

# Deep sort imports
from deep_sort import nn_matching

import config

%matplotlib inline 


In [2]:
from cav.detection import ObjectDetector
import cv2

# Here should be a patch to your object detection model. 

od = ObjectDetector(config.MODEL_PATH)

#### Parameters

In [ ]:
CUT_UPPER = 0
SAVE_DETECTIONS = os.path.join(config.DATA_PATH, 'detections.p')
VIDEO_FILE = pickle.load(open(f'{config.DATA_PATH}/videopath.p', 'rb'))

# Create folders
Path(os.sep.join(SAVE_DETECTIONS.split(os.sep)[:-1])).mkdir(parents=True, exist_ok=True)



In [ ]:
cap = cv2.VideoCapture(VIDEO_FILE)

objects = []

results = []
colors = {}


save_detections = {}    

i = 0
t = time()

while cap.isOpened():
    video_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
    video_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - CUT_UPPER     
    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_timeStamp = i/fps

    t2 = time() - t
    sys.stdout.write('{} frames done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
        i, t2, i/t2))                   
    i += 1
    
    ret, image = cap.read()
    if image is None:
        break

    if CUT_UPPER > 0:
        image = image[CUT_UPPER:, :, :]
    
    boxes, scores, classes = od.detect(image, timestamp=frame_timeStamp)
    if SAVE_DETECTIONS is not None:
        save_detections[i] = (boxes, scores, classes)
    
cap.release()

if SAVE_DETECTIONS is not None:
    pickle.dump(save_detections, open(SAVE_DETECTIONS,'wb'))
    print (f'\nDetections saved in {SAVE_DETECTIONS}.')



In [ ]:
print ('({}x{}), {:.1f} fps'.format(video_x, video_y, fps))